In [27]:
using Distributions,Plots,DataFrames;
plotlyjs();

In [2]:
using StatsBase

In [3]:
using Measures

## Bayesian Estimation for Linear Regression Problem

We have linear regression model $y_i = \beta_0 + \beta_1 x_i + \epsilon_i$, where $\epsilon_i \sim \mathcal{N}(0,\phi)$ and suppose we observe the data $(y_i,x_i), i \in \{1,2,\cdots,n\}$, then our model $y$ is $y_{i} \sim \mathcal{N}(\beta_0 + \beta_1 x_i,\phi), i = 1,2,\cdots,n$.
Our interest is making inference about $\beta_0$ and $\beta_1$. If we consider normal priors on the coefficients and an inverse gamma prior on the variance term, then full Bayesian model foe this data can be written as $y_i |\beta_0,\beta_1,\phi \sim \mathcal{N}(\beta_0 + \beta_1 x_i , \phi), i = 1,2,\cdots, n.\\
\beta_0|\mu_0,\tau_0 \sim \mathcal{N}(\mu_0,\tau_0)\\
\beta_1|\mu_1,\tau_1 \sim \mathcal{N}(\mu_1,\tau_1)\\
\phi|\alpha,\gamma \sim \mathcal{IG}(\alpha,\gamma)$,

where 

$\mu_0,\tau_0,\mu_1,\tau_1,\alpha,\beta$ 

all are hyper-parameters and we assume that they are known.

Aloso assume that $\beta_0,\beta_1,\phi$ all are independent variables, so their joint distribution $f(\beta_0,\beta_1,\phi)$ acn be written as $f(\beta_0,\beta_1,\phi) = f(\beta_0).f(\beta_1).f(\phi)$.

Manually, we calculated all the posterior distribution for $\beta_0,\beta_1,\phi$, for $\beta_0,\beta_1$, we didn't get the posterior as a known distribution but for varinace $\phi$, we get Inverse Gamma distribution with shape $\alpha + \frac{n}{2}$ and scale $ \frac{1}{2}\sum_{i =1}^{n}(y_i - \beta_0 - \beta_1 x_i)^2 +\gamma$.

In [4]:
beta_0 = 3 #population intercept
beta_1 = 1 # population slope
phi = 1 #population error variance
x = LinRange(1,5,50) #generating 50 elements from 1 to 5 including both

using Random;
Random.seed!(123)
y = beta_0 .+ beta_1.*x + rand(Normal(0,sqrt(phi)),length(x))

scatter(x,y,c = 3,label = :none) #plotting the data points

In [5]:
# prior for beta_0

mu_0 = 2;sigma_0 = 0.4

function prior_beta_0(x)
    pdf(Normal(mu_0,sqrt(sigma_0)),x)
end

# prior for beta_1

mu_1 = 2;sigma_1 = 0.5;

function prior_beta_1(x)
    pdf(Normal(mu_1,sqrt(sigma_1)),x)
end

# prior for phi

a = 2;b = 2;
function prior_phi(x)
    pdf(InverseGamma(a,(1/b)x))#shape 'a' and scale 'b'
end

prior_phi (generic function with 1 method)

In [6]:
# likelihood function of the parameters
function likeli(x,y,params)
    beta_0 = params[1]
    beta_1 = params[2]
    phi = params[3]
    n = length(x)
    log_likelihhod = -n*log(sqrt(2*pi)) - (n/2)*log(phi) -sum((y .- beta_0 .- beta_1*x).^2)/(2*phi)
    # log likelihood function 
    return exp(log_likelihhod)
end

likeli (generic function with 1 method)

In [7]:
m = 100000 #number of iterations
using Random;
Random.seed!(1234)
post_beta_0 = []
post_beta_1 = []
post_phi =[]

# as we want to apply grid approximation approach,so we need to choose grid step
step = 0.01  #grid step

grid_beta_0 = Vector(-5:step:10)

grid_beta_1 = Vector(-5:step:10)

push!(post_beta_0,rand(Normal(mu_0,sqrt(sigma_0)),1)[1])
#initialization of the posterior  value of beta_0

push!(post_beta_1,rand(Normal(mu_1,sqrt(sigma_1)),1)[1])
#initialization of the posterior value of beta_1

push!(post_phi,rand(InverseGamma(a,(b)),1)[1]);
#intialization of posterior value of phi

By initialization the chains, we want to create a long chain  which converge to a stationary distribution(which is nothing but posterior) after a burning-period. To do that we mainly use Gibbs sampler method but there is a problem with Gibbs is that, for applying we need to know the marginal distribution. But here we have no idea about the joint posterior distribution,so the exact from of marginal is also not possible. That's why we need to apply grid approximation mehtod.

In [8]:

for i in 2:m
    
    # section for posterior sample of beta_0
    tmp_post_beta_0 = []
    for j in 1:length(grid_beta_0)
        params = [grid_beta_0[j],post_beta_1[i-1],post_phi[i-1]]
        push!(tmp_post_beta_0,likeli(x,y,params)*prior_beta_0(grid_beta_0[j]))
    end
    prob = tmp_post_beta_0./ sum(tmp_post_beta_0)
    push!(post_beta_0,sample(grid_beta_0,Weights(prob)))
    #sample of posterior beta_0 after normalize 
    
    
    # section for posterior sample of beta_1
    tmp_post_beta_1 = []
    for j in 1:length((grid_beta_1))
        params = [post_beta_0[i],grid_beta_1[j],post_phi[i-1]]
        push!(tmp_post_beta_1,likeli(x,y,params)*prior_beta_1(grid_beta_1[j]))
    end
    probs = tmp_post_beta_1 ./ sum(tmp_post_beta_1)
    push!(post_beta_1,sample(grid_beta_1,Weights(probs)))
    # sample of posterior beta_1 after normalize
    
    # section for posterior sample of phi
    shape = a + length(x)/2
    rate = (1/2)*sum((y .- post_beta_0[i] .- post_beta_1[i]*x).^2) .+ b
    
    push!(post_phi,rand(InverseGamma(shape,(rate)),1)[1])
    # sample of posterior phi
end   

Trace plots are useful tools to converge of the chains which are expected to converge the posterior distribution. After an initial burn in period, the samples act as samples from the target(desired distribution). We consider here our burn in period as $70000$ i.e. $70\%$ of the total iteration.

In [29]:
p1 = plot(post_beta_0,c = 2,label = "trace plot of beta_0")
p2 = plot(post_beta_1,c = 3,label = "trace plot of beta_1")
p3 = plot(post_phi,c = 4,label = "trace plot of phi");

In [31]:
plot(p1,p2,p3)
savefig("lin reg 1")

Noted that, simulation of ${\beta_0}^{(j)}$ depends on the value of ${\beta_0}^{(j-1)}$. Simialrly for other parameters also. Thus the posterior values are identically distributed(after a sufficiently large step) but not independent. So, the test for autocorrelation has been performed as the values has been selected accordingly i.e. consider every $16$th value after burn in period.This process is known as thinning. 

To find the autocorrelation, use 'autocor' fuction , then plotted the autocor values it shows that the thinning value we are choosing it allows us that after thinning  what are chain values we are getting they are independent .

In [11]:
cut = 0.5

burn_in = Int(m*cut)

u = post_beta_0[burn_in:end];

In [12]:
index = Vector(1:16:length(u))

post_beta_0_thining = u[index];
post_beta_0_thining = convert(Vector{Float16},post_beta_0_thining)

autocor(post_beta_0_thining);

u = post_beta_1[burn_in:end]

post_beta_1_thining = convert(Vector{Float16},u[index])
autocor(post_beta_1_thining)

u = post_phi[burn_in:end]
post_phi_thinning = convert(Vector{Float16},u[index])
autocor(post_phi_thinning);

In [32]:
p = plot(autocor(post_beta_0_thining),xlabel = "lags",ylabel = "acf",title = "acf of posterior beta_0",label = :none );
p1 = plot(autocor(post_beta_1_thining),xlabel ="lags",ylabel = "acf",title = "acf of posterior of beta_1",label = :none);
p2 = plot(autocor(post_phi_thinning),xlabel ="lags",ylabel = "acf",title = "acf of posterior of phi",label = :none);


In [33]:
plot(p,p1,p2,margin = 10mm)
savefig("lin reg 2")

After thinning , we visualize the approximated distribution by means of histogram. Plotted the histogram of posterior samples for $\beta_0,\beta_1,\phi$ and the overlay the curve of normal distribution $\mathcal{N}({\mu_1}',{\sigma_1}')$,${\mu_1}' =$ mean of`post_beta_0_thinning`, ${\sigma_1}' = $ variance of `post_beta_0_thinning`,$\mathcal{N}({\mu_2}',{\sigma_2}')$,${\mu_2}' =$ mean of `post_beta_1_thinning`, ${\sigma_2}' = $ variance of `post_beta_1_thinning`, and the curve of Inverse Gamma with shape = $\alpha + \frac{n}{2}$, scale = $\frac{1}{2}\sum_{i =1}^{n}(y_i - \beta_0 - \beta_1 x_i)^2 +\gamma$ respectively.


In [34]:
p = histogram(post_beta_0_thining,normalize = true,c = 3,
    label = "posterior of beta_0")

credible_interval = quantile(post_beta_0_thining,(2.5,97.5)./100)
p1 = vline!([credible_interval[1],credible_interval[2]],linewidth = 3,c = 2, 
label = "crdible interval for beta 0")
plot!(0:0.01:4,pdf(Normal(mean(post_beta_0_thining),sqrt(var(post_beta_0_thining))),0:0.01:4),
    linewidth = 3,c = "blue",style = :dash,label = "N(2.447,0.10724)")
savefig("lin reg 3")

In [35]:
p2 = histogram(post_beta_1_thining,normalize = true,c = 4,
    label = "posterior of beta_1")
credible_interval = quantile(post_beta_1_thining,(2.5,97.5)./100)
p3 = vline!([credible_interval[1],credible_interval[2]],linewidth = 3,c = 1, 
label = "crdible interval for beta 1")
plot!(0.7:0.01:1.5,pdf(Normal(mean(post_beta_1_thining),sqrt(var(post_beta_1_thining))),0.7:0.01:1.5),
linewidth = 2,style = :dot,c = "red",label = "N(1.1045,0.01182)")
savefig("lin reg 4")

In [37]:
p = histogram(post_phi_thinning,normalize = true,c = 4,
    label = "posterior of phi")
credible_interval = quantile(post_phi_thinning,(2.5,97.5)./100)

vline!([credible_interval[1],credible_interval[2]],linewidth = 3,c = 5, 
label = "crdible interval for phi")
# savefig("lin reg 5")

In [38]:
mean_post_beta_0 = mean(post_beta_0_thining)
mean_post_beta_1 = mean(post_beta_1_thining)
shape = a + length(x)/2

scale = (1/2)*sum((y .- mean_post_beta_0 .- mean_post_beta_1*x).^2) + b

plot!(0:0.1:2,pdf(InverseGamma(shape,(scale)),0:0.1:2),c = 6,label = "IG(shape,scale)",linewidth = 3,style = :dash)
savefig("lin reg 5")

In [19]:
mean_post_beta_0
# this is approximated value of beta_0 whose original value is 3

Float16(2.502)

In [20]:
mean_post_beta_1
# this is approximated value of beta_1 whose original value is 1

Float16(1.11)

In [21]:
var(post_beta_0_thining)

Float16(0.10077)

In [22]:
var(post_beta_1_thining)

Float16(0.011055)

In [23]:
x = LinRange(1,5,50) #generating 50 elements from 1 to 5 including both

using Random;
Random.seed!(123)
y = beta_0 .+ beta_1.*x + rand(Normal(0,sqrt(phi)),length(x))
scatter(x,y,label ="data",c = 3)

In [24]:
x = LinRange(1,5,50)
y_fit = mean_post_beta_0 .+ mean_post_beta_1.*x + rand(Normal(0,sqrt(mean(post_phi))),length(x))
plot!(x,y_fit,c = 5,style = :dash,linewidth = 3)